In [456]:
import pandas as pd
import numpy as np
import re

# Load file
with open("modaction.txt", "r", encoding="utf-8") as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]  # Remove empty lines

data = []
i = 0

# Process lines in pairs (username + action)
while i < len(lines) - 1:
    username = lines[i]
    action = lines[i + 1]

    # Check if the second line is actually an action
    if "by" in action.lower() or "acknowledged" in action.lower():
        data.append([username, action])
        i += 2  # Move to the next user-action pair
    else:
        i += 1  # Skip messages that aren't actions

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Username", "Action"])

# Extract useful info if needed
df["Moderator"] = df["Action"].str.extract(r"by (\w+)")
df["Reason"] = df["Action"].str.extract(r"with reason: (.+?)(?:\s•|$)")
df["Time"] = df["Action"].str.extract(r"• (.+)")


In [457]:
df.head(10)

,Username,Action,Moderator,Reason,Time
0,z1dorov,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,3 hours ago
1,s_ramazan077,Message Deleted by balintboss • 3 hours ago,balintboss,NaN,3 hours ago
2,NoraExplorer,Raid Started by dorozea with 2261 viewers • 3 ...,dorozea,NaN,3 hours ago
3,Josephs_x,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
4,Josephs_x,Message Deleted by Fossabot • 3 hours ago,Fossabot,NaN,3 hours ago
5,inn_________,Acknowledged their warning • 3 hours ago,NaN,NaN,3 hours ago
6,inn_________,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
7,palamdorzhievroman,Acknowledged their warning • 3 hours ago,NaN,NaN,3 hours ago
8,palamdorzhievroman,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
9,larrik,Acknowledged their warning • 3 hours ago,NaN,NaN,3 hours ago


In [458]:
df.describe()

,Username,Action,Moderator,Reason,Time
count,3907,3907,3162,1896,3900
unique,2075,524,15,21,15
top,unknown user,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago
freq,23,264,2261,890,899


In [459]:
df = df[~df['Username'].isin([  'Slow Mode',
                                'Subs-Only Chat',
                                'Emotes-Only Chat',
                                'Followers-Only Chat'
                            ])]
df = df[~df['Action'].str.startswith((  'Acknowledged',
                                        'Added to Suspicious User monitoring',
                                        'Unban request',
                                        'Removed as Blocked Term',
                                        'Added as Blocked Term',
                                        'Shouted out',
                                        'Added as Permitted Term',
                                        'Invited',
                                        'Added as moderator'
                                    ), na=False)]

In [460]:
df.head(10)

,Username,Action,Moderator,Reason,Time
0,z1dorov,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,3 hours ago
1,s_ramazan077,Message Deleted by balintboss • 3 hours ago,balintboss,NaN,3 hours ago
2,NoraExplorer,Raid Started by dorozea with 2261 viewers • 3 ...,dorozea,NaN,3 hours ago
3,Josephs_x,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
4,Josephs_x,Message Deleted by Fossabot • 3 hours ago,Fossabot,NaN,3 hours ago
6,inn_________,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
8,palamdorzhievroman,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
10,larrik,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,3 hours ago
11,larrik,Message Deleted by Fossabot • 3 hours ago,Fossabot,NaN,3 hours ago
12,djavathano,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,3 hours ago


In [461]:
df.loc[df['Time'].str.contains('hours', na=False), 'Time'] = '0'

In [462]:
df.loc[df['Action'].str.endswith('yesterday', na=False), 'Time'] = '1'

In [463]:
df.tail(10)

,Username,Action,Moderator,Reason,Time
3895,pepper_blast,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9 days ago
3898,baldrix_008,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9 days ago
3899,baldrix_008,Message Deleted by Fossabot • 9 days ago,Fossabot,NaN,9 days ago
3900,namezhas,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago
3901,SioNaaR,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago
3902,wdomq,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago
3903,kef1r10,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9 days ago
3904,hadya_23,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago
3905,aracubedoto,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9 days ago
3906,whysalizz,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9 days ago


In [464]:
df['Time'] = df['Time'].str.extract(r'(\d+)')

In [465]:
df.tail(25)

,Username,Action,Moderator,Reason,Time
3877,manas72088,"Timed out by Fossabot for 144000 seconds, with...",Fossabot,please move on - automated by Fossabot,9
3878,sutnays,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9
3879,psychokity1,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9
3881,FrezZzy13,"Timed out by Fossabot for 60 seconds, with rea...",Fossabot,Phrase (#6) - automated by Fossabot,9
3882,duffyrus,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9
3883,duffyrus,Message Deleted by Fossabot • 9 days ago,Fossabot,NaN,9
3884,nortveizer,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9
3885,09akadil,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9
3888,FrezZzy13,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9
3889,sinanpolat1,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,9


In [466]:
conditions = [
    df['Action'].str.startswith('Warned', na=False),
    df['Action'].str.startswith('Timed out', na=False),
    df['Action'].str.startswith('Timeout removed', na=False),
    df['Action'].str.startswith('Message Deleted', na=False),
    df['Action'].str.startswith('Raid Started', na=False),
    df['Action'].str.startswith('Banned', na=False),
    df['Action'].str.startswith('Unbanned', na=False)
    
]

choices = ['warning', 'timeout', 'unTimeout','delete','raid','ban','unban']

df['actionType'] = np.select(conditions, choices, default='other')

In [467]:
df = df[df['actionType'] != 'other']

In [468]:
df.head(10)

,Username,Action,Moderator,Reason,Time,actionType
0,z1dorov,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,0,timeout
1,s_ramazan077,Message Deleted by balintboss • 3 hours ago,balintboss,NaN,0,delete
2,NoraExplorer,Raid Started by dorozea with 2261 viewers • 3 ...,dorozea,NaN,0,raid
3,Josephs_x,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,0,warning
4,Josephs_x,Message Deleted by Fossabot • 3 hours ago,Fossabot,NaN,0,delete
6,inn_________,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,0,warning
8,palamdorzhievroman,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,0,warning
10,larrik,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,0,warning
11,larrik,Message Deleted by Fossabot • 3 hours ago,Fossabot,NaN,0,delete
12,djavathano,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,0,timeout


In [469]:
df.describe()

,Username,Action,Moderator,Reason,Time,actionType
count,3066,3066,3066,1896,3066,3066
unique,1996,450,14,21,10,7
top,unknown user,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,9,timeout
freq,20,264,2258,890,733,1610


In [470]:
df[df['Username']=='unknown user']

,Username,Action,Moderator,Reason,Time,actionType
1194,unknown user,Banned by Aluminiumminimumimmunity • 4 days ago,Aluminiumminimumimmunity,NaN,4,ban
1225,unknown user,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,4,warning
1631,unknown user,Timed out by balintboss for 600000 seconds • 5...,balintboss,NaN,5,timeout
1636,unknown user,Timed out by balintboss for 345000 seconds • 5...,balintboss,NaN,5,timeout
1750,unknown user,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,6,timeout
1977,unknown user,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,6,timeout
1997,unknown user,Banned by balintboss • 6 days ago,balintboss,NaN,6,ban
2001,unknown user,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,6,warning
2324,unknown user,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,7,timeout
2460,unknown user,"Timed out by Fossabot for 72000 seconds, with ...",Fossabot,please move on - automated by Fossabot,7,timeout


In [471]:


# Extract the number of seconds from 'Action' where 'actionType' is 'timeout'
df['timeOutTime'] = df.apply(lambda row: int(re.search(r'(\d+) seconds', row['Action']).group(1)) 
                             if row['actionType'] == 'timeout' and pd.notna(row['Action']) and re.search(r'(\d+) seconds', row['Action']) 
                             else None, axis=1)

In [472]:
df[df['Time']=='2'].head(25)

,Username,Action,Moderator,Reason,Time,actionType,timeOutTime
874,4eburek3457,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,2,timeout,345600.0
875,nihplod999,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,2,timeout,345600.0
876,bradymanek,Raid Started by dorozea with 1648 viewers • 2 ...,dorozea,NaN,2,raid,NaN
877,ppuuudi,"Timed out by Fossabot for 345600 seconds, with...",Fossabot,please move on - automated by Fossabot,2,timeout,345600.0
878,xsantiiagoo,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,2,warning,NaN
879,xsantiiagoo,Message Deleted by Fossabot • 2 days ago,Fossabot,NaN,2,delete,NaN
880,freakpro_uhne,"Timed out by Fossabot for 600 seconds, with re...",Fossabot,English Only In Chat so mods can moderate bett...,2,timeout,600.0
882,freakpro_uhne,"Warned by Fossabot, with reason: English Only ...",Fossabot,English Only In Chat so mods can moderate bett...,2,warning,NaN
883,RELlKT,Unbanned by Martin_Gales • 2 days ago,Martin_Gales,NaN,2,unban,NaN
884,shalun66,Timed out by balintboss for 86400 seconds • 2 ...,balintboss,NaN,2,timeout,86400.0


In [473]:
timeout = df[df['actionType']=='timeout']

In [474]:
timeout.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1610 entries, 0 to 3906
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Username     1610 non-null   object 
 1   Action       1610 non-null   object 
 2   Moderator    1610 non-null   object 
 3   Reason       1124 non-null   object 
 4   Time         1610 non-null   object 
 5   actionType   1610 non-null   object 
 6   timeOutTime  1397 non-null   float64
dtypes: float64(1), object(6)
memory usage: 100.6+ KB


In [475]:
timeout = timeout.drop(columns=['actionType', 'Reason', 'Action'])


In [476]:
timeout['hours'] =timeout['timeOutTime'] / 60 / 60

In [477]:
timeout.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1610 entries, 0 to 3906
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Username     1610 non-null   object 
 1   Moderator    1610 non-null   object 
 2   Time         1610 non-null   object 
 3   timeOutTime  1397 non-null   float64
 4   hours        1397 non-null   float64
dtypes: float64(2), object(3)
memory usage: 75.5+ KB


In [478]:
# 1. Count how many timeouts each moderator has
mod_counts = timeout.groupby('Moderator').size()

# 2. Sum of hours for each moderator
mod_hours_sum = timeout.groupby('Moderator')['hours'].sum()

# 3. Statistical measures for each moderator's timeout hours
mod_stats = timeout.groupby('Moderator')['hours'].agg([
    ('median', 'median'),
    ('mean', 'mean'),
    ('min', 'min'),
    ('max', 'max')
])

# Combine all statistics into one dataframe
mod_stats_combined = pd.DataFrame({
    'timeout': mod_counts,
    'timeout_in_hours': mod_hours_sum
}).join(mod_stats)


In [479]:
mod_stats_combined

,timeout,timeout_in_hours,median,mean,min,max
Moderator,,,,,,
1206paul_,31,182.666667,1.000000,6.298851,0.083333,24.000000
55Dash,5,337.016667,0.500000,67.403333,0.016667,168.000000
Aloddin,1,0.001944,0.001944,0.001944,0.001944,0.001944
Aluminiumminimumimmunity,169,1331.400000,1.000000,24.207273,0.016667,168.000000
Fossabot,1110,44886.829167,20.000000,40.475049,0.002778,192.000000
Hueqi,3,216.000000,24.000000,72.000000,24.000000,168.000000
Ivana_10,33,637.461667,0.166667,19.317020,0.016667,193.602500
Martin_Gales,36,1719.017222,1.000000,53.719288,0.000556,336.000000
StreamElements,5,0.027778,0.005556,0.005556,0.005556,0.005556


In [480]:
# Get the list of all unique moderators from both dataframes
all_moderators = pd.concat([
    pd.Series(timeout['Moderator'].unique()),
    pd.Series(df['Moderator'].unique())
]).unique()

# Create action counts for all moderators
filtered_df = df[df['actionType'].isin(['ban', 'warning', 'delete'])]
action_counts = pd.crosstab(filtered_df['Moderator'], filtered_df['actionType'], dropna=False)

# Ensure all action types exist as columns
for action in ['ban', 'warning', 'delete']:
    if action not in action_counts.columns:
        action_counts[action] = 0

# For moderators not in the action_counts dataframe, add them with zeros
missing_mods = [mod for mod in all_moderators if mod not in action_counts.index]
for mod in missing_mods:
    action_counts.loc[mod] = [0, 0, 0]  # Add zeros for ban, warning, delete

# For the original statistics, we'll need to handle moderators not in timeout dataframe
# First, recreate the original statistics
mod_counts = timeout.groupby('Moderator').size()
mod_hours_sum = timeout.groupby('Moderator')['hours'].sum()
mod_stats = timeout.groupby('Moderator')['hours'].agg([
    ('median', 'median'),
    ('mean', 'mean'),
    ('min', 'min'),
    ('max', 'max')
])

# Combine into mod_stats_combined
mod_stats_combined = pd.DataFrame({
    'timeout_count': mod_counts,
    'total_hours': mod_hours_sum
}).join(mod_stats)

# Create a complete result dataframe with all moderators
result = pd.DataFrame(index=all_moderators)

# Join with the timeout statistics, filling NAs with 0 for counts and hours
result = result.join(mod_stats_combined, how='left')
result['timeout_count'] = result['timeout_count'].fillna(0).astype(int)
result['total_hours'] = result['total_hours'].fillna(0)

# For the statistical measures, we'll keep NAs as they correctly indicate "no data"
# Now join with the action counts
result = result.join(action_counts, how='left')
result[['ban', 'warning', 'delete']] = result[['ban', 'warning', 'delete']].fillna(0).astype(int)

# Display the results
print(result)

                          timeout_count   total_hours     median       mean  \
Fossabot                           1110  44886.829167  20.000000  40.475049   
Aluminiumminimumimmunity            169   1331.400000   1.000000  24.207273   
balintboss                          113   4125.133056   0.500000  44.838403   
1206paul_                            31    182.666667   1.000000   6.298851   
banties_x                            99    671.583333   0.166667  23.985119   
Ivana_10                             33    637.461667   0.166667  19.317020   
StreamElements                        5      0.027778   0.005556   0.005556   
klimzaa                               3     24.169444   0.166667   8.056481   
55Dash                                5    337.016667   0.500000  67.403333   
Martin_Gales                         36   1719.017222   1.000000  53.719288   
gkey                                  2     24.166667  12.083333  12.083333   
Hueqi                                 3    216.00000

In [481]:
result

,timeout_count,total_hours,median,mean,min,max,ban,delete,warning
Fossabot,1110,44886.829167,20.000000,40.475049,0.002778,192.000000,0,378,770
Aluminiumminimumimmunity,169,1331.400000,1.000000,24.207273,0.016667,168.000000,58,0,0
balintboss,113,4125.133056,0.500000,44.838403,0.002500,324.782222,50,16,5
1206paul_,31,182.666667,1.000000,6.298851,0.083333,24.000000,10,1,0
banties_x,99,671.583333,0.166667,23.985119,0.016667,168.000000,22,0,0
Ivana_10,33,637.461667,0.166667,19.317020,0.016667,193.602500,9,42,0
StreamElements,5,0.027778,0.005556,0.005556,0.005556,0.005556,0,0,0
klimzaa,3,24.169444,0.166667,8.056481,0.002778,24.000000,4,0,0
55Dash,5,337.016667,0.500000,67.403333,0.016667,168.000000,4,0,0
Martin_Gales,36,1719.017222,1.000000,53.719288,0.000556,336.000000,19,5,2


In [482]:
result = result.drop(columns=['median', 'mean', 'min', 'max'])

In [483]:
result

,timeout_count,total_hours,ban,delete,warning
Fossabot,1110,44886.829167,0,378,770
Aluminiumminimumimmunity,169,1331.400000,58,0,0
balintboss,113,4125.133056,50,16,5
1206paul_,31,182.666667,10,1,0
banties_x,99,671.583333,22,0,0
Ivana_10,33,637.461667,9,42,0
StreamElements,5,0.027778,0,0,0
klimzaa,3,24.169444,4,0,0
55Dash,5,337.016667,4,0,0
Martin_Gales,36,1719.017222,19,5,2
